# MI2RL Deep Learning Hands-on (2): Medical Image Classification

![alt text](https://adeshpande3.github.io/assets/Cover.png)

# 0. 시작에 앞서... 

Menu -> Runtime -> Change runtime type 

![gpu setting](https://raw.githubusercontent.com/mi2rl/datasets/master/gpu.png)



# 1. 데이터 준비: MedNIST dataset

In [0]:
# 데이터 다운로드
!wget https://raw.githubusercontent.com/mi2rl/datasets/master/mednist.tar.gz

In [0]:
# 압축 풀기
!tar xzf mednist.tar.gz

In [0]:
# 패키지 불러오기
import numpy as np
import os
import time
%matplotlib inline
import matplotlib.pyplot as plt
from PIL import Image


In [0]:
dataDir = 'resized'               # 데이터 위치
classNames = os.listdir(dataDir)  # 각 클래스의 이름들
numClass = len(classNames)        # Number of classes = number of subdirectories
imageFiles = [[os.path.join(dataDir,classNames[i],x) for x in os.listdir(os.path.join(dataDir,classNames[i]))]
              for i in range(numClass)]                     # 각 클래스 별 파일 이름들
numEach = [len(imageFiles[i]) for i in range(numClass)]     # 각 클래스 별 파일 갯수
imageFilesList = []               # 모든 파일이름
imageClass = []                   # 각각의 파일들에 대한 클래스 

for i in range(numClass):
    imageFilesList.extend(imageFiles[i])
    imageClass.extend([i]*numEach[i])
    
numTotal = len(imageClass)        # 전체 파일 갯수
imageWidth, imageHeight = Image.open(imageFilesList[0]).size         # 각 영상의 사이즈(width, height)

print("There are",numTotal,"images in",numClass,"distinct categories")
print("Label names:",classNames)
print("Label counts:",numEach)
print("Image dimensions:",imageWidth,"x",imageHeight)

In [0]:
# 전체 이미지 중 9개를 랜덤으로 골라 3x3으로 레이블과 함께 그리기
# -- 여러번 실행하며 이미지들을 살펴보세요 --

plt.subplots(3,3,figsize=(8,8))
for i,k in enumerate(np.random.randint(numTotal, size=9)): 
    im = Image.open(imageFilesList[k])                      
    arr = np.array(im)
    plt.subplot(3,3,i+1)
    plt.xlabel(classNames[imageClass[k]])
    plt.imshow(arr,cmap='gray',vmin=0,vmax=255)
plt.tight_layout()
plt.show()

In [0]:
# 이미지 리스트 살펴보기
imageFilesList[0:10]

# 2. VGG16를 이용한 분류 실습 (w/ ImageNet pre-trained weight)

![VGG16 네트워크 구조](https://www.cs.toronto.edu/~frossard/post/vgg16/vgg16.png)

In [0]:
import keras
from keras.layers import Input, Conv2D, MaxPool2D, Dense, Flatten
from keras.models import Model

# 2.1. [Quiz] 순서가 섞인 layer들을 VGG16 구성에 맞게 배치해보세요.

In [0]:
inputs = Input(shape=(224, 224, 3,), name="VGGInput")

In [0]:
x = Conv2D(filters=64, kernel_size=(3,3), padding='same', activation=relu)(inputs)
x = Conv2D(filters=64, kernel_size=(3,3), padding='same', activation=relu)(x)
x = MaxPool2D(padding='same')(x)

In [0]:
x = Conv2D(filters=512, kernel_size=(3,3), padding='same', activation=relu)(x)
x = Conv2D(filters=512, kernel_size=(3,3), padding='same', activation=relu)(x)
x = Conv2D(filters=512, kernel_size=(3,3), padding='same', activation=relu)(x)
x = MaxPool2D(padding='same')(x)

In [0]:
x = Conv2D(filters=128, kernel_size=(3,3), padding='same', activation=relu)(x)
x = Conv2D(filters=128, kernel_size=(3,3), padding='same', activation=relu)(x)
x = MaxPool2D(padding='same')(x)

In [0]:
x = Conv2D(filters=256, kernel_size=(3,3), padding='same', activation=relu)(x)
x = Conv2D(filters=256, kernel_size=(3,3), padding='same', activation=relu)(x)
x = Conv2D(filters=256, kernel_size=(3,3), padding='same', activation=relu)(x)
x = MaxPool2D(padding='same')(x)

In [0]:
x = Conv2D(filters=4096, kernel_size=(7,7), padding='valid', activation=relu)(x)
x = Flatten()(x)
x = Dense(4096, activation=relu)(x)
pred = Dense(1000, activation=softmax)(x)

In [0]:
x = Conv2D(filters=512, kernel_size=(3,3), padding='same', activation=relu)(x)
x = Conv2D(filters=512, kernel_size=(3,3), padding='same', activation=relu)(x)
x = Conv2D(filters=512, kernel_size=(3,3), padding='same', activation=relu)(x)
x = MaxPool2D(padding='same')(x)

In [0]:
model = Model(inputs=inputs, outputs=pred)

In [0]:
model.summary()

In [0]:
optimizer = Adam(learning_rate=1e-03)
model.compile(optimizer=optimizer, loss='categorical_crossentropy')

# 2.2. VGG16 모델 불러오기

In [0]:
from keras.applications import vgg16

# VGG16 모델 불러오기
model = vgg16.VGG16()

# 모델의 모양을 보여준다.
model.summary()

In [0]:
# Model 구성도 plot
from IPython.display import Image

keras.utils.plot_model(model, to_file='vgg16.png', show_shapes=True, show_layer_names=True)
Image(filename='vgg16.png')


**VGG16**

`keras.applications.vgg16.VGG16(include_top=True, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=1000)`


VGG16 model, with weights pre-trained on ImageNet.

This model can be built both with 'channels_first' data format (channels, height, width) or 'channels_last' data format (height, width, channels).

The default input size for this model is 224x224.

**Arguments**



*   include_top: whether to include the 3 fully-connected layers at the top of the network.
*   weights: one of None (random initialization) or 'imagenet' (pre-training on ImageNet).
*   input_tensor: optional Keras tensor (i.e. output of layers.Input()) to use as image input for the model.
*   input_shape: optional shape tuple, only to be specified if include_top is False (otherwise the input shape has to be (224, 224, 3) (with 'channels_last' data format) or (3, 224, 224) (with 'channels_first' data format). It should have exactly 3 inputs channels, and width and height should be no smaller than 32. E.g. (200, 200, 3) would be one valid value.
*   pooling: Optional pooling mode for feature extraction when include_top is False.
*   classes: optional number of classes to classify images into, only to be specified if include_top is  True, and if no weights argument is specified.

# keras 에서 제공되는 모델들 참고: https://keras.io/applications/

In [0]:
# VGG16 모델을 이용해 prediction 하는 함수
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from IPython.display import display # 이미지 출력 함수

def predict_vgg16(model, filename) :
    # 이미지 파일을 읽고 화면에 표시
    image = load_img(filename)
    display(image)

    # 모델 사이즈로 이미지 파일을 읽기
    image = load_img(filename, target_size=(224, 224))

    # 이미지 데이터를 numpy로 변환
    image = img_to_array(image)

    # vgg16.preprocess_input()을 호출하기 위해 차원을 조정
    # 보통 모델을 여러 이미지를 한번에 호출. 
    # 맨 앞의 1 : 이미지 갯수가 1개라는 것.
    # 두번째 224 : 가로
    # 세번째 224 : 세로
    # 네번째 3 : R, G, B 3개
    image = image.reshape((1, 224, 224, 3))

    # VGG16 모델 호출을 위해 데이터 전처리.
    # -255 ~ 255 사이 값으로 정규화한다.
    # 그리고 RGB를 BGR순으로 바꾼다.
    image = vgg16.preprocess_input(image)


    # 이미지를 모델에 적용
    yhat = model.predict(image)

    # 모델 적용된 결과를 파싱
    label = vgg16.decode_predictions(yhat)

    # 가장 확률이 높은 결과를 획득
    label = label[0][0]

    # 라벨과 라벨을 예측한 확률을 출력
    print('%s (%.2f%%)' % (label[1], label[2]*100))    

In [0]:
files = imageFilesList[0:10]

In [0]:
for file in files:
  predict_vgg16(model, file)

# 2.3. Dataset 나누기: Train / Validation / Test

In [0]:
validFrac = 0.2   # Define the fraction of images to move to validation dataset
testFrac = 0.2    # Define the fraction of images to move to test dataset
validList = []
testList = []
trainList = []

for i in range(numTotal):
    rann = np.random.random() # Randomly reassign images
    if rann < validFrac:
        validList.append(i)
    elif rann < testFrac + validFrac:
        testList.append(i)
    else:
        trainList.append(i)
        
nTrain = len(trainList)  # Count the number in each set
nValid = len(validList)
nTest = len(testList)
print("Training images =",nTrain,"Validation =",nValid,"Testing =",nTest)

In [0]:
!mkdir ./train
!mkdir ./valid
!mkdir ./test

In [0]:
import shutil
from tqdm import tqdm

for i in tqdm(range(len(trainList))):
  root, clas, src = imageFilesList[trainList[i]].split('/')
  dest = os.path.join('./train',clas,src)
  if not os.path.exists(os.path.join('./train',clas)):
    os.mkdir(os.path.join('./train',clas))
  shutil.copy(imageFilesList[trainList[i]], dest)
  
for i in tqdm(range(len(validList))):
  root, clas, src = imageFilesList[validList[i]].split('/')
  dest = os.path.join('./valid',clas,src)
  if not os.path.exists(os.path.join('./valid',clas)):
    os.mkdir(os.path.join('./valid',clas))
  shutil.copy(imageFilesList[validList[i]], dest)
  
    
for i in tqdm(range(len(testList))):
  root, clas, src = imageFilesList[testList[i]].split('/')
  dest = os.path.join('./test',clas,src)
  if not os.path.exists(os.path.join('./test',clas)):
    os.mkdir(os.path.join('./test',clas))
  shutil.copy(imageFilesList[testList[i]], dest)


# 2.4. Image Data Generator 정의 (+Data Augmentation)

In [0]:
from keras.preprocessing.image import ImageDataGenerator

train_dir = './train'
validation_dir = './valid'
batch_size = 32
image_size = 224

# 학습에 사용될 이미지 데이터 생성기
train_datagen = ImageDataGenerator(
      rotation_range=180, # 회전 최대 20도
      width_shift_range=0.2, # 좌우 이동
      height_shift_range=0.2, # 상하 이동
      horizontal_flip=True, # 좌우 반전
      vertical_flip=True, # 상하 반전
      )
 
# 검증에 사용될 이미지 데이터 생성기
validation_datagen = ImageDataGenerator()
 

# 학습에 사용될 데이터 생성기  
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(image_size, image_size),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=True)

# 검증에 사용될 데이터 생성기
validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=(image_size, image_size),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False)

class_num=len(train_generator.class_indices)

custom_labels = list(validation_generator.class_indices.keys())

In [0]:
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras.models import Model
from keras import models
from keras import layers
from keras import optimizers
from keras.applications import VGG16
import keras.backend as K

K.clear_session() # 새로운 세션으로 시작

# 2.5. VGG16 Fine-tuning

In [0]:
# 모델 불러오기
conv_layers = VGG16(weights='imagenet', include_top=False, input_shape=(image_size, image_size, 3))
conv_layers.summary()

# Convolution Layer를 학습되지 않도록 고정 
for layer in conv_layers.layers:
    layer.trainable = False

In [0]:
# 새로운 모델 생성하기
model = models.Sequential()

# VGG16모델의 Convolution Layer를 추가
model.add(conv_layers)
 
# 모델의 Fully Connected 부분을 재구성
model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(class_num, activation='softmax'))

In [0]:
# 새로운 모델 요약
model.summary()

In [0]:
# 새로운 모델 저장
vgg16_model_path = 'new_trained_from_vgg16.h5'

model.save(vgg16_model_path)

# 2.6. 새로운 모델 학습

In [0]:
from keras.models import load_model

# 모델 로딩
model = load_model(vgg16_model_path)

# 모델 컴파일
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])

# 모델 학습
history = model.fit_generator(
      train_generator,
      steps_per_epoch=100 ,
      epochs=10,
      validation_data=validation_generator,
      validation_steps=validation_generator.samples/validation_generator.batch_size,
      verbose=1)
 
# 모델 저장
model.save(vgg16_model_path)

# 2.7. 학습 결과 시각화

In [0]:
acc = history.history['acc']
loss = history.history['loss']
valacc = history.history['val_acc']
valloss = history.history['val_loss']


epochs = range(len(acc))
 
plt.plot(epochs, acc, 'b', label='accuracy')
plt.plot(epochs, loss, 'r', label='loss')
plt.plot(epochs, valacc, 'b--', label='val_accuracy')
plt.plot(epochs, valloss, 'r--', label='val_loss')
plt.title('accuracy and loss')
plt.legend()
  
plt.show()

# 2.8. 학습된 모델을 이용해 Test 데이터에 대한 Prediction

In [0]:
test_dir = './test'

In [0]:
test_datagen = ImageDataGenerator()
test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(image_size, image_size),
        batch_size=1,
        class_mode='categorical',
        shuffle=False)

model.evaluate_generator(test_generator, steps=test_generator.samples, verbose=1)

# 3. VGG16를 이용한 분류 실습 (from Scratch)

In [0]:
# 모델 불러오기
vgg_model = VGG16(weights=None, include_top=False, input_shape=(image_size, image_size, 3))
vgg_model.summary()

In [0]:
last = vgg_model.output

In [0]:
# VGG16모델에 Fully Connected부분을 재구성해서 추가
x = Flatten()(last)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
pred = Dense(class_num, activation='softmax')(x)

model = Model(vgg_model.input, pred)

In [0]:
# 새로운 모델 요약
model.summary()

In [0]:
# 새로운 모델 저장
vgg16_model_path = 'new_trained_from_vgg16_scratch.h5'

model.save(vgg16_model_path)

In [0]:
# 모델 로딩
model = load_model(vgg16_model_path)

# 모델 컴파일
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])

# 모델 학습
history = model.fit_generator(
      train_generator,
      steps_per_epoch=100 ,
      epochs=5,
      validation_data=validation_generator,
      validation_steps=validation_generator.samples/validation_generator.batch_size,
      verbose=1)
 
# 모델 저장
model.save(vgg16_model_path)

In [0]:
acc = history.history['acc']
loss = history.history['loss']
valacc = history.history['val_acc']
valloss = history.history['val_loss']


epochs = range(len(acc))
 
plt.plot(epochs, acc, 'b', label='accuracy')
plt.plot(epochs, loss, 'r', label='loss')
plt.plot(epochs, valacc, 'b--', label='val_accuracy')
plt.plot(epochs, valloss, 'r--', label='val_loss')
plt.title('accuracy and loss')
plt.legend()
  
plt.show()

In [0]:
test_dir = './test'

test_datagen = ImageDataGenerator()
test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(image_size, image_size),
        batch_size=1,
        class_mode='categorical',
        shuffle=False)
# For evaluation purposes
#model.evaluate_generator(test_generator, steps=test_generator.samples, verbose=1)

# For prediction purposes
y_pred = model.predict_generator(test_generator, steps=test_generator.samples, verbose=1)

In [0]:
y_pred.shape

In [0]:
y_pred0 = np.argmax(y_pred, axis=1)

In [0]:
y_pred0

# 4. Confusion Matrix Visualization

In [0]:
import pandas as pd
import seaborn as sn
from sklearn.metrics import confusion_matrix, classification_report

In [0]:
# y_test labeling
y_test = test_generator.labels

In [0]:
# calculate confusion matrix for the predicted dataset
cm = confusion_matrix(y_test, y_pred0)

In [0]:
# make a dataframe using cm array
df_cm = pd.DataFrame(cm, index = [i for i in classNames], columns = [i for i in classNames])

In [0]:
# plot confusion matrix
plt.figure(figsize = (10, 7))
sn.heatmap(df_cm, annot=True)

In [0]:
# classification report generation: precision, recall, f1-score. 
print(classification_report(y_test, y_pred0, target_names=classNames))

# 5. VGG16 + GradCAM

![gradCAM](https://camo.githubusercontent.com/450498bd998fd99d51b647d2b6c8631e94585522/687474703a2f2f692e696d6775722e636f6d2f4a614762645a352e706e67)

**Grad-CAM: Why did you say that? Visual Explanations from Deep Networks via Gradient-based Localization**
Ramprasaath R. Selvaraju, Abhishek Das, Ramakrishna Vedantam, Michael Cogswell, Devi Parikh, Dhruv Batra
https://arxiv.org/abs/1610.02391

**Example: 'Boxer'**

![alt text](https://github.com/PowerOfCreation/keras-grad-cam/raw/master/examples/cat_dog.png) 
![alt text](https://github.com/PowerOfCreation/keras-grad-cam/raw/master/examples/cat_dog_242_gradcam.jpg)
![alt text](https://github.com/PowerOfCreation/keras-grad-cam/raw/master/examples/cat_dog_242_guided_gradcam.jpg)

In [0]:
# GradCAM code from https://github.com/PowerOfCreation/keras-grad-cam/blob/master/grad-cam.py

from keras.applications.vgg16 import (
    VGG16, preprocess_input, decode_predictions)
from keras.models import Model
from keras.preprocessing import image
from keras.layers.core import Lambda
from keras.models import Sequential
from keras.models import load_model
from tensorflow.python.framework import ops
import keras.backend as K
import tensorflow as tf
import numpy as np
import keras
import sys
import cv2
import random

def target_category_loss(x, category_index, nb_classes):
    return tf.multiply(x, K.one_hot([category_index], nb_classes))

def target_category_loss_output_shape(input_shape):
    return input_shape

def normalize(x):
    # utility function to normalize a tensor by its L2 norm
    return x / (K.sqrt(K.mean(K.square(x))) + 1e-5)

def load_images(path):
    img_path = sys.argv[1]
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return x

def register_gradient():
    if "GuidedBackProp" not in ops._gradient_registry._registry:
        @ops.RegisterGradient("GuidedBackProp")
        def _GuidedBackProp(op, grad):
            dtype = op.inputs[0].dtype
            return grad * tf.cast(grad > 0., dtype) * \
                tf.cast(op.inputs[0] > 0., dtype)

def compile_saliency_function(model, activation_layer='block5_conv3'):
    input_img = model.input
    layer_dict = dict([(layer.name, layer) for layer in model.layers[1:]])
    layer_output = layer_dict[activation_layer].output
    max_output = K.max(layer_output, axis=3)
    saliency = K.gradients(K.sum(max_output), input_img)[0]
    return K.function([input_img, K.learning_phase()], [saliency])

def modify_backprop(model, name):
    g = tf.get_default_graph()
    with g.gradient_override_map({'Relu': name}):

        # get layers that have an activation
        layer_dict = [layer for layer in model.layers[1:]
                      if hasattr(layer, 'activation')]

        # replace relu activation
        for layer in layer_dict:
            if layer.activation == keras.activations.relu:
                layer.activation = tf.nn.relu

        # re-instanciate a new model
        new_model = VGG16(weights='imagenet')
    return new_model

def deprocess_image(x):
    '''
    Same normalization as in:
    https://github.com/fchollet/keras/blob/master/examples/conv_filter_visualization.py
    '''
    if np.ndim(x) > 3:
        x = np.squeeze(x)
    # normalize tensor: center on 0., ensure std is 0.1
    x -= x.mean()
    x /= (x.std() + 1e-5)
    x *= 0.1

    # clip to [0, 1]
    x += 0.5
    x = np.clip(x, 0, 1)

    # convert to RGB array
    x *= 255
    if K.image_dim_ordering() == 'th':
        x = x.transpose((1, 2, 0))
    x = np.clip(x, 0, 255).astype('uint8')
    return x

def _compute_gradients(tensor, var_list):
    grads = tf.gradients(tensor, var_list)
    return [grad if grad is not None else tf.zeros_like(var) for var, grad in zip(var_list, grads)]

def grad_cam(input_model, image, category_index):
    nb_classes = 6
    target_layer = lambda x: target_category_loss(x, category_index, nb_classes)
    x = Lambda(target_layer, output_shape = target_category_loss_output_shape)(input_model.output)
    model = Model(inputs=input_model.input, outputs=x)
    loss = K.sum(model.output)
    
    conv_output =  model.layers[17].output
    grads = normalize(_compute_gradients(loss, [conv_output])[0])
    gradient_function = K.function([model.input], [conv_output, grads])

    output, grads_val = gradient_function([image])
    output, grads_val = output[0, :], grads_val[0, :, :, :]

    weights = np.mean(grads_val, axis = (0, 1))
    cam = np.ones(output.shape[0 : 2], dtype = np.float32)

    for i, w in enumerate(weights):
        cam += w * output[:, :, i]

    cam = cv2.resize(cam, (224, 224))
    cam = np.maximum(cam, 0)
    heatmap = cam / np.max(cam)

    #Return to BGR [0..255] from the preprocessed image
    image = image[0, :]
    image -= np.min(image)
    image = np.minimum(image, 255)

    cam = cv2.applyColorMap(np.uint8(255*heatmap), cv2.COLORMAP_JET)
    cam = np.float32(cam) + np.float32(image)
    cam = 255 * cam / np.max(cam)
    return np.uint8(cam), heatmap

In [0]:
vgg16_model_path = 'new_trained_from_vgg16_scratch.h5'
model = load_model(vgg16_model_path)

In [0]:
model.summary()

In [0]:
# Select a random query image
inum = random.randrange(0,len(imageFilesList))
qimage = load_img(imageFilesList[inum], target_size=(224, 224))
qimage = img_to_array(qimage)
qimage = qimage.reshape((1, 224, 224, 3))

In [0]:
predictions = model.predict(qimage)
predicted_class = np.argmax(predictions)

In [0]:
# Grad-CAM Calculation & Visualization
cam, heatmap = grad_cam(model, qimage, predicted_class)
plt.title(classNames[predicted_class])
plt.imshow(cam)

In [0]:
register_gradient()
guided_model = modify_backprop(model, 'GuidedBackProp')
saliency_fn = compile_saliency_function(guided_model)
saliency = saliency_fn([qimage, 0])
gradcam = saliency[0] * heatmap[..., np.newaxis]
plt.imshow(gradcam[0])